## **Atención Q, K y V**

Con el objetivo de entender con detalle el funcionamiento de la atención en transformers, implementaremos una versión simplificada de la atención con los vectores Q, K y V.

In [2]:
import torch

# Supongamos que tenemos los siguiente vectores Q, K y V
Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

print(Q.shape)
print(K.shape)
print(V.shape)

score = Q @ K.transpose(0, 1)  # @ equivale a la multiplicación de matrices

print("\nScore:\n", score)

score = score / torch.sqrt(torch.tensor(K.shape[1]).float())  # Dividimos por la raíz cuadrada de la dimensión de K
score = torch.softmax(score, dim=1)

print("\nScore normalizado:\n", score)

Z = score @ V  

print("\nResultado:\n", Z)

torch.Size([4, 3])
torch.Size([4, 3])
torch.Size([4, 3])

Score:
 tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.3000, 0.6000, 0.9000, 1.2000],
        [0.0600, 0.1200, 0.1800, 0.2400],
        [0.0900, 0.1800, 0.2700, 0.3600]])

Score normalizado:
 tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [0.1892, 0.2250, 0.2676, 0.3182],
        [0.2372, 0.2455, 0.2542, 0.2631],
        [0.2309, 0.2432, 0.2561, 0.2698]])

Resultado:
 tensor([[0.2500, 0.5000, 0.5000],
        [0.1892, 0.5432, 0.5857],
        [0.2372, 0.5087, 0.5173],
        [0.2309, 0.5130, 0.5260]])


### **Todo en uno: scaled_dot_product_attention**

Todo el cálculo anterior lo realiza eficientemente una función de PyTorch llamada **scaled_dot_product_attention()**. Esta función calcula la atención en los tensores de consulta (query), clave (key) y valor (value), utilizando una máscara de atención opcional si se proporciona, y aplicando *dropout* si se especifica una probabilidad mayor a 0.0.

https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html

In [56]:
from torch.nn.functional import scaled_dot_product_attention

Z = scaled_dot_product_attention(Q, K, V)

print(Z)

tensor([[0.2500, 0.5000, 0.5000],
        [0.1892, 0.5432, 0.5857],
        [0.2372, 0.5087, 0.5173],
        [0.2309, 0.5130, 0.5260]])


## **Atención Q, K y V con enmascaramiento**

El enmascaramiento durante la etapa del decodificador en los modelos Transformer es crucial para evitar que el decodificador tenga acceso a información futura, especialmente en tareas de generación secuencial como la traducción automática o la generación de texto. Este concepto se conoce como "enmascaramiento de atención causal".

En el contexto de los Transformers, el decodificador genera una salida secuencialmente, palabra por palabra. Durante la generación de cada palabra, es importante que el modelo solo tenga en cuenta las palabras anteriores y no las futuras, ya que estas últimas no deberían estar disponibles (en un escenario de generación de texto, por ejemplo, las palabras futuras aún no se han generado).

In [9]:
import torch

Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

score = Q @ K.transpose(0, 1)
score = score / torch.sqrt(torch.tensor(K.shape[1]).float())
print("\nScore:\n", score)
tril = torch.tril(torch.ones(score.shape[0], score.shape[0]))  # Creamos la máscara
score_masked = score.masked_fill(tril == 0, float('-inf'))  # Aplicamos la máscara al score. Todo lo que sea 0 en la máscara, lo reemplazamos por -inf para que al aplicar la softmax, se vuelva 0
print("\nScore masked:\n", score_masked)
score_masked = torch.softmax(score_masked, dim=1)
print("\nScore masked with softmax:\n", score_masked)
Z = score_masked @ V 
print("\nZ:\n", Z) 


Score:
 tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.1732, 0.3464, 0.5196, 0.6928],
        [0.0346, 0.0693, 0.1039, 0.1386],
        [0.0520, 0.1039, 0.1559, 0.2078]])

Score masked:
 tensor([[0.0000,   -inf,   -inf,   -inf],
        [0.1732, 0.3464,   -inf,   -inf],
        [0.0346, 0.0693, 0.1039,   -inf],
        [0.0520, 0.1039, 0.1559, 0.2078]])

Score masked with softmax:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000, 0.0000],
        [0.3219, 0.3332, 0.3449, 0.0000],
        [0.2309, 0.2432, 0.2561, 0.2698]])

Z:
 tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])


Vemos de nuevo que podemos hacer lo mismo con la función **torch.nn.functional.scaled_dot_product_attention()** estableciendo el parámetro **is_causal** a True.

In [54]:
Z = scaled_dot_product_attention(Q, K, V, is_causal=True)

print(Z)

tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])


## **Layer normalization**

---

### Ejercicio 2: Implementación de la normalización de capa

Asumiento los parámetros $\gamma$ y $\beta$ como 1 y 0 respectivamente, desarrolla un código que normalice los siguientes vectores de características:

In [50]:
import torch

v1 = torch.tensor([1.0, 2.1, 3.0])
v2 = torch.tensor([2.0, 2.0, 2.0])
v3 = torch.tensor([3.0, 1.9, 1.0])

# Inicializamos listas para guardar los vectores normalizados
v1_normalized_manual = torch.zeros_like(v1)
v2_normalized_manual = torch.zeros_like(v2)
v3_normalized_manual = torch.zeros_like(v3)

# Lista de vectores para iterar más fácilmente
vectors = [v1, v2, v3]

# Calculamos manualmente la normalización de capa para cada vector
for i, v in enumerate(vectors):
    mean = v.mean()
    var = v.var(unbiased=False)  # Calculamos la varianza sin sesgo
    std = torch.sqrt(var + 1e-12)  # Añadimos epsilon para estabilidad numérica
    vectors[i] = (v - mean) / std  # Normalizamos

# Asignamos los vectores normalizados a sus respectivas variables
v1_normalized_manual, v2_normalized_manual, v3_normalized_manual = vectors

print(v1_normalized_manual)
print(v2_normalized_manual)
print(v3_normalized_manual)

# Para verificar, usamos la implementación de PyTorch de Layer Normalization
from torch.nn.functional import layer_norm

# Agrupamos los vectores originales para aplicar layer_norm
v = torch.stack([v1, v2, v3])

# Aplicamos layer_norm a todo el conjunto de vectores
#v_normalized_pytorch = layer_norm(v, v.shape[1:], eps=1e-12)
v_normalized_pytorch = layer_norm(v, v.shape, eps=1e-12)

print(v_normalized_pytorch)



tensor([-1.2635,  0.0815,  1.1820])
tensor([0., 0., 0.])
tensor([ 1.2635, -0.0815, -1.1820])
tensor([[-1.4963e+00,  1.4963e-01,  1.4963e+00],
        [ 1.7837e-07,  1.7837e-07,  1.7837e-07],
        [ 1.4963e+00, -1.4963e-01, -1.4963e+00]])
torch.Size([3, 3])
torch.Size([3])


In [45]:
import numpy as np

# Crear una población de alturas (en cm) de 100 personas
# np.random.seed(42)  # Semilla para reproducibilidad
# poblacion_alturas = np.random.normal(loc=170, scale=10, size=100)  # media=170, desviación estándar=10
poblacion_alturas = np.random.uniform(low=150, high=190, size=100)  # uniforme entre 150 y 190

# Varianza poblacional
varianza_poblacional = np.var(poblacion_alturas)

# Tomar 1000 muestras diferentes de tamaño 5 de la población
n_muestras = 1000
tamaño_muestra = 5
varianzas_muestrales_con_sesgo = []
varianzas_muestrales_sin_sesgo = []

for _ in range(n_muestras):
    muestra = np.random.choice(poblacion_alturas, tamaño_muestra, replace=False)
    varianzas_muestrales_con_sesgo.append(np.var(muestra, ddof=0))
    varianzas_muestrales_sin_sesgo.append(np.var(muestra, ddof=1))

# Calcular el promedio de las varianzas muestrales
promedio_var_muestrales_con_sesgo = np.mean(varianzas_muestrales_con_sesgo)
promedio_var_muestrales_sin_sesgo = np.mean(varianzas_muestrales_sin_sesgo)

varianza_poblacional, promedio_var_muestrales_con_sesgo, promedio_var_muestrales_sin_sesgo


(114.47514973310177, 89.81829275768195, 112.27286594710245)